In [12]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

import time
# tmp sleep
# time.sleep(3600)
print('sleep done')
for f in sorted(glob.glob('../features/*.pkl')):
    print("'{}',".format(f))

sleep done
'../features/cnn_muse_1_feat.pkl',
'../features/cnn_muse_adj_1_feat.pkl',
'../features/gru_muse_1_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_1_feat.pkl',
'../features/lstm_2_feat.pkl',
'../features/lstm_muse_1_feat.pkl',
'../features/lstm_muse_adj_1_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/other_feat.pkl',
'../features/simple_rnn_muse_1_feat.pkl',
'../features/tfidf_feat1.pkl',
'../features/tfidf_feat2.pkl',


In [17]:
# load feats
chosen_fl = [
    '../features/cnn_muse_1_feat.pkl',
    '../features/lstm_muse_1_feat.pkl',
    '../features/cnn_muse_adj_1_feat.pkl',
    '../features/lstm_muse_adj_1_feat.pkl',
    #'../features/lstm_2_feat.pkl',
    #'../features/lstm_1_feat.pkl',
    '../features/gru_muse_1_feat.pkl',
    '../features/simple_rnn_muse_1_feat.pkl',
    '../features/other_feat.pkl',
]
train_x,test_x = [],[]
for feat in chosen_fl:
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/simple_rnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
(95851, 52)
(95851, 52)


In [18]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [19]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [20]:
res = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = res
sample_submission.to_csv("../results/xgb_some_ens_csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0856394068358 0.0876596106445
1
0.0183819315759 0.0226964529779
2
0.0459994420958 0.0473688267919
3
0.00507859999759 0.00899363247159
4
0.0563034884333 0.0631216386809
5
0.0164251624111 0.0195272241891
this fold avg train 0.0379713385582 avg val 0.0415612309593
0
0.0839202063059 0.0917113682987
1
0.0184128926591 0.0222273936628
2
0.0443975288747 0.0507953690184
3
0.00543616926195 0.00855259110158
4
0.0572084749792 0.0605895985445
5
0.0161233496318 0.0205361670686
this fold avg train 0.0375831036188 avg val 0.0424020812825
0
0.0840473143681 0.0911879374837
1
0.0183304644918 0.0227042477312
2
0.044209562436 0.0519885934284
3
0.00572753943419 0.00735698906043
4
0.0569121250882 0.0614976209301
5
0.0162421827788 0.0203828278788
this fold avg train 0.0375781980995 avg val 0.0425197027521
all train avg 0.0377108800922 all val avg 0.042161004998
         id     toxic  severe_toxic   obscene    threat    insult  \
0   6044863  0.002497      0.000042  0.000688  0.000112  0.000446   
1   6102